In [1]:
from transformers import CLIPProcessor, CLIPVisionModelWithProjection
import torch, pathlib
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "openai/clip-vit-large-patch14" 
processor  = CLIPProcessor.from_pretrained(model_name)
vision_enc = CLIPVisionModelWithProjection.from_pretrained(model_name).to(device)
vision_enc.eval()

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

CLIPVisionModelWithProjection(
  (vision_model): CLIPVisionTransformer(
    (embeddings): CLIPVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (position_embedding): Embedding(257, 1024)
    )
    (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-23): 24 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=1024, out_fe

In [2]:
print(f"device {device}")

device cuda


In [1]:
#1 frame per second from video
import cv2
import os
import numpy as np

video_path = "../movie.mp4"
frames_dir = "../feature_extraction/frames"
os.makedirs(frames_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = int(np.ceil(frame_count / fps))

print(f"number of frames: {int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}")
print(f"frame rate: {cap.get(cv2.CAP_PROP_FPS)}")
print(f"length in seconds: {cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)}")

for sec in range(duration):
    cap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)
    success, image = cap.read()
    if success:
        cv2.imwrite(os.path.join(frames_dir, f"frame_{sec:04d}.jpg"), image)#put each frame in frame folder
    else:
        print(f"Frame at {sec}s not saved")


cap.release()

number of frames: 87720
frame rate: 24.0
length in seconds: 3655.0


In [4]:
import sys, os
sys.path.append(os.path.abspath(".."))
from feature_extraction.clip_extractor import initialize_clip_model, extract_clip_features, save_clip_features

#clip model
processor, vision_model = initialize_clip_model()
frame_indices = np.arange(duration)
features = extract_clip_features(frames_dir, frame_indices, processor, vision_model)
save_clip_features(features, "vision_clip_features.mat")


Extracting CLIP features: 100%|█████████████| 3655/3655 [04:12<00:00, 14.45it/s]


Final CLIP feature shape: (3655, 768)
Saved features to vision_clip_features.mat
